# Requirments
- transformer: huggingface transformers
- datasets: klue dataset load
- evaluate: 데이터셋 evaluate

In [1]:
!pip install transformers

!pip install transformers[torch]
!pip install accelerate -U
# Python 프로그램에서 데이터셋을 쉽게 다룰 수 있게 해주는 datasets 라이브러리를 설치하는 작업
!pip install datasets

!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00


# Import

In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cuda')

# Data Load
- klue 데이터셋에서 연합뉴스 토픽 분류 라벨링 된 데이터(TC a.k.a ynat)
- 파인튜닝할 데이터셋

In [4]:
# Hugging Face의 datasets 라이브러리를 사용하여 'klue'와 'ynat' 데이터셋을 불러와 datasets라는 변수에 저장
datasets = load_dataset('klue', 'ynat') # 'klue' 데이터셋에서 'ynat' 부분집합을 불러오기 (데이터셋의 이름과 데이터셋의 구체적인 설정(config))
                                        # Yes/No/N/A Type Classification - 네 아니요 모름 세 가지 클래스로 분류

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [5]:
# 데이터셋 내에 어떻게 들어있나 보기 위한 함수

# 임의의 예제를 선택하여 출력하는 함수를 정의
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

show_random_elements(datasets["validation"])

,guid,title,label,url,date
0,ynat-v1_dev_05035,우리 집엔 DJ가 산다…발달장애 오빠를 둔 여대생의 이야기,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=250&oid=001&aid=0011862059,2020.09.05. 오전 6:10
1,ynat-v1_dev_07104,주파수 대역정비 예보제 도입…5G 주파수 늘린다,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0011711034,2020.06.29. 오후 4:06
2,ynat-v1_dev_07143,그래픽 줄기세포 이식 파킨슨병에 효과 연구 모식도,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0009510717,2017.08.31. 오전 2:00
3,ynat-v1_dev_03123,프로배구 KB손보 베테랑 센터 김형우 영입,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010282107,2018.08.20 17:27
4,ynat-v1_dev_02608,K스마일 캠페인 일본 오모테나시와 만나다,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=237&oid=001&aid=0009633595,2017.10.25. 오후 2:26
5,ynat-v1_dev_02543,옛 방식 그대로 고대 무덤 대형옹관 만들어볼까,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0010143310,2018.06.12. 오전 9:33
6,ynat-v1_dev_04360,원격수업만으론 충분한 교육제공 못해…등교재개 검사후 결정,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=250&oid=001&aid=0011636924,2020.05.27. 오후 12:00
7,ynat-v1_dev_02197,근로복지공단 이사장에 강순희 씨 임명…24일 취임,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=251&oid=001&aid=0011417108,2020.02.21. 오전 11:01
8,ynat-v1_dev_02663,게시판 신문윤리위 온라인신문 대표 세미나 개최,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011632888,2020.05.25. 오후 7:18
9,ynat-v1_dev_00480,19승 투수 세베리노 양키스와 4년 장기계약,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010638238,2019.02.16 10:05


https://huggingface.co/monologg/kobigbird-bert-base

In [6]:
pretrained_model = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [7]:
# 연습용
test_model = AutoModel.from_pretrained(pretrained_model)
test_model

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0-11): 12 x BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdBlockSparseAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
    

In [8]:
sentences = ['안녕하세요 hmkd 여러분', '무더운 여름에 건강관리 잘하세요!']
features = tokenizer(
    sentences,
    max_length=10,
    padding='max_length',
    truncation=True,
)
features

{'input_ids': [[2, 9279, 4681, 9421, 576, 4522, 4507, 4565, 8280, 3], [2, 26787, 7775, 4584, 7206, 11066, 3747, 4681, 9421, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
features = {k:torch.tensor(v) for k,v in features.items()}
output_tensor = test_model(**features)
print(output_tensor)

Attention type 'block_sparse' is not possible if sequence_length: 10 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0085,  0.3430, -0.4673,  ...,  0.1736, -0.2570,  0.0558],
         [ 0.4125, -0.6818, -0.1634,  ...,  0.0846,  0.1364,  0.4318],
         [ 0.4620,  0.0325,  0.0777,  ...,  0.1734,  0.0707,  0.1572],
         ...,
         [-0.2735,  0.2188,  0.1350,  ..., -0.2034,  0.2622,  0.4731],
         [ 0.1283,  0.2342, -0.3019,  ..., -0.2352,  0.1830,  0.6579],
         [-0.0145,  0.0533,  0.2154,  ..., -0.1981, -0.0600,  0.3875]],

        [[ 0.3253, -0.0567, -0.1669,  ...,  0.0151,  0.3085,  0.1041],
         [ 0.0719, -0.0903,  0.0267,  ..., -0.2625,  0.7100,  0.1628],
         [-0.0882, -0.0213, -0.1715,  ..., -0.1045,  0.8717,  0.2090],
         ...,
         [ 0.1878, -0.0069, -0.2244,  ...,  0.2413,  0.4385, -0.0058],
         [ 0.1836, -0.0750,  0.2445,  ...,  0.2398,  0.3535, -0.1645],
         [-0.1313, -0.2876, -0.0321,  ..., -0.2087,  0.4650,  0.2791]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [10]:
for key, value in output_tensor.items():
    print(f"Size of tensor '{key}': {value.size()}")

Size of tensor 'last_hidden_state': torch.Size([2, 10, 768])
Size of tensor 'pooler_output': torch.Size([2, 768])


In [11]:
# 데이터 tokenize
def preprocess_function(examples):
    return tokenizer(examples['title'], truncation=True, return_token_type_ids=False,) # 토큰의 타입 정보를 반환하지 않도록 함
# preprocess_function은 데이터셋의 각 예제에 대해 토큰화 작업을 수행하고, 토큰화된 결과를 새로운 데이텃ㅅ에 저장, batched=True로 설정되어 있으
tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date', 'input_ids', 'attention_mask'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date', 'input_ids', 'attention_mask'],
        num_rows: 9107
    })
})

In [13]:
import evaluate

accuracy = evaluate.load('accuracy')

In [14]:
# 기사의 본문을 입력으로 받아서 해당 기사가 7개 topic 중 어느 것에 속하는지를 분류
# loss(accuracy) 적용
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: 'IT과학', 1: '경제', 2: '사회', 3: '생활문화', 4: '세계', 5: '스포츠', 6: '정치'}
label2id = {'IT과학': 0, '경제': 1,'사회': 2, '생활문화': 3, '세계': 4, '스포츠': 5, '정치': 6}

# model define
- AutoModelForSequenceClassification.from_pretrained: 이용하면 쉽게 모델 정의 가능
- https://huggingface.co/docs/transformers/model_doc/auto

In [15]:
# SequenceClassification에 좀 더 특화된 model로 사용해 보기
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=7, id2label=id2label, label2id=label2id)

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

# Argument set & Run
- 여기서 모델이 쓰이는 parameter 정의
- 만들어진 모델은 huggingface에(온라인) 저장한 후 불러올 수 있음
- 불러와서 사용할 때 pipeline으로 사용 가능

In [17]:
training_args = TrainingArguments(
    output_dir='topic_model',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 18 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.378200,0.383379,0.871198


TrainOutput(global_step=2855, training_loss=0.46409706610096735, metrics={'train_runtime': 350.5892, 'train_samples_per_second': 130.289, 'train_steps_per_second': 8.143, 'total_flos': 495932362009212.0, 'train_loss': 0.46409706610096735, 'epoch': 1.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.3833785653114319,
 'eval_accuracy': 0.8711979795761502,
 'eval_runtime': 15.1992,
 'eval_samples_per_second': 599.176,
 'eval_steps_per_second': 37.502,
 'epoch': 1.0}

In [19]:
# Save the model manually
# config.json 및 pytorch_model.bin 파일이 'my_model' 디렉토리에 생성
model.save_pretrained('topic_model')

# 문장 하나에 적용

In [20]:
text = '국회, 2월 임시회 개회...6~8일 대정부질문 여야 주자는?'
inputs = tokenizer(text, truncation=True, return_token_type_ids=False, return_tensors='pt').to(device)
model.eval()    # 모델을 평가 모드로 설정
outputs = model(**inputs)
predictions = outputs.logits.argmax(-1) # argmax(-1)는 마지막 차원(일반적으로 클래스 차원)을 따라 가장 큰 값의
id2label[predictions.cpu().numpy()[0]] # 예측된 클래스 인덱스를 넘파이 배열로 변환하고 그 주 첫번째 값을 레이블로 변경


'정치'

In [21]:
!mkdir /content/drive/MyDrive/project_task/AI한글교육솔루션/foreigners/best_model

mkdir: cannot create directory ‘/content/drive/MyDrive/project_task/AI한글교육솔루션/foreigners/best_model’: No such file or directory


In [22]:
!Is

/bin/bash: line 1: Is: command not found


In [ ]:
!cp -r ./topic_model ./content/drive/MyDrive/코딩/hmkd1/best_model

cp: cannot create directory './content/drive/MyDrive/코딩/hmkd1/best_model': No such file or directory


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 대회 데이터에 적용해보기

# Requirements
- transformer: huggingface transformers

In [ ]:
!pip install transformers
# Python 프로그램에서 데이터셋을 쉽게 다룰 수 있게 해주는 datasets 라이브러리를 설치하는 작업
!pip install datasets

!pip install evaluate

# model define

In [ ]:
# SequenceClassification에 좀 더 특화된 model로 사용해 보기
model = AutoModelForSequenceClassification.from_pretrained("위 모델 돌려서 저장한 경로통로")

In [ ]:
model

# 문장 하나에 적용

In [ ]:
def inference_fn(sentence):
  inputs = tokenizer(sentence, truncation=True, return_token_ids=False, return_tensers='pt').to(device)
  model.eval()
  outputs = model(**inputs)
  predictions = outputs.logits.argmax(-1)  # argmax(-1)는 마지막 차원(일반적으로 클래스 차원)을 따라 가장 큰 값의 인덱스를 반환

In [ ]:
sentence = "대통령은 국회에서 국정연설을 한다."
inference_fn(sentence)

#### 웹 구현

In [ ]:
!mkdir /root/.ngrok2 && echo "ngrok에서 받은 authtoken 인증토큰" > /root/.ngrok2/ngrok.yml

ngrok에서 받은 authtoken 인증토큰


In [ ]:
!pip install flask_ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template

def get_web_service_app(inference_fn, is_colab=True):   # colab 환경에서 실행 시 외부에서 접근 가능한 url을 생성

    app = Flask(__name__, template_folder='')   # Flask 애플리케이션 생성. __name__은 현재 모듈의 이름
    if is_colab:
        from flask_ngrok import run_with_ngrok
        run_with_ngrok(app)
    else:
        from flask_cors import CORS
        CORS(app)

    @app.route('/') # 경로에 대한 라우트를 정의
    def index():
        return render_template('index.html')    # index.html 파일을 랜더링

    @app.route('/api', methods=['POST'])    # 이 라우트는 POST 메서드를 사용 -> 코드로 넘김(긴 내용일 경우) <-> get은 주소창에 직접 넘김
    def api():
        query_sentence = request.json['sentence']   # api 함수는 클라이언트로부터 JSON 형식의 데이터를 받아옴
        output_data = inference_fn(query_sentence)
        response = jsonify(output_data) # OUTPUT 데이터를 JSO?N 형식으로 변환하여 클라이언트에게 반환
        return response

    return app  # Flask 객체를 반환

In [ ]:
from google.colab import files

uploade = files.upload()